#  Import libaries

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


# Load Dataset

In [6]:
dataset = pd.read_csv(r"D:\PROJECTS\Git\HMI_MODEL_PROTHESIS_ARM\Signals\dataset.csv")

data= dataset.sample(frac=1, random_state=42).reset_index(drop=True)

mean = data['Force/Output'].mean()
std_dev = data['Force/Output'].std()
z_scores = np.abs((data['Force/Output'] - mean) / std_dev)
print(z_scores)


# Define a threshold for identifying outliers
threshold = 3  

# Identify outliers
outliers = (z_scores > threshold) | (z_scores < -threshold)

# Remove outliers
data_outl = data[~outliers]

# Show the results
print("Original dataset shape:", data.shape)
print("No outliers dataset shape:", data_outl.shape)


0        0.801318
1        0.291351
2        1.159545
3        0.917972
4        0.801318
           ...   
51673    0.280378
51674    0.801318
51675    0.801318
51676    0.801318
51677    0.801318
Name: Force/Output, Length: 51678, dtype: float64
Original dataset shape: (51678, 8)
No outliers dataset shape: (51311, 8)


# Extract features and target

In [7]:
# Load your dataset
x = data[['Ch1_RMS', 'Ch1_MAV', 'Ch1_ZC', 'Ch2_RMS', 'Ch2_MAV', 'Ch2_ZC','Label']]
y = data['Force/Output']

# Split dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Normalize data
scale = StandardScaler()
x_train = scale.fit_transform(x_train)
x_test = scale.transform(x_test)

In [8]:
# Objective function for SSA (Minimizing MSE)
def objective_function(params):
    C, epsilon = params
    model = SVR(kernel='rbf', C=C, epsilon=epsilon, gamma='auto')
    model.fit(x_train, y_train)
    predictions = model.predict(x_test)
    mse = mean_squared_error(y_test, predictions)
    return mse

# SSA Algorithm for hyperparameter optimization
def SSA(n_spiders=15, max_iter=1,patience=15):
    # Define hyperparameter ranges
    C_range = (0.1, 100)  # C range
    epsilon_range = (0.1, 1)  # epsilon range

    # Initialize spiders (hyperparameters)
    spiders = np.zeros((n_spiders, 2))
    spiders[:, 0] = np.random.uniform(*C_range, n_spiders)  # C values
    spiders[:, 1] = np.random.uniform(*epsilon_range, n_spiders)  # epsilon values

    best_mse = float('inf')
    best_params = None

    for iteration in range(max_iter):
        for i in range(n_spiders):
            mse = objective_function(spiders[i])
            if mse < best_mse:
                best_mse = mse
                best_params = spiders[i]
                no_improvement_count = 0  # Reset if improvement occurs
            else:
                no_improvement_count += 1

            # Update spider positions (simplified)
            spiders[i] += np.random.uniform(-1, 1)  # Random movement
            # Ensure the values stay within defined ranges
            spiders[i][0] = np.clip(spiders[i][0], *C_range)
            spiders[i][1] = np.clip(spiders[i][1], *epsilon_range)

        # Check for early stopping
        if no_improvement_count >= patience:
            print(f"Early stopping at iteration {iteration} with best MSE: {best_mse}")
            break

    return best_params, best_mse

# Run SSA for SVR hyperparameter optimization
optimal_params, optimal_mse = SSA()

# Output optimal parameters
print(f"Optimal Parameters (C, epsilon): {optimal_params}")
print(f"Optimal Mean Squared Error: {optimal_mse}")

# Fit SVR model with the optimal parameters
final_model = SVR(kernel='rbf', C=optimal_params[0], epsilon=optimal_params[1], gamma='auto')
final_model.fit(x_train, y_train)

# Predictions and evaluation
final_predictions = final_model.predict(x_test)

r2 = r2_score(y_test, final_predictions)
mse = mean_squared_error(y_test, final_predictions)
mae = mean_absolute_error(y_test, final_predictions)

# Output results
print(f"R² Score: {r2}")
print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")

Optimal Parameters (C, epsilon): [98.63115736  0.55550044]
Optimal Mean Squared Error: 6.8181535142947745
R² Score: 0.7372742656597191
Mean Squared Error: 6.828423890297631
Mean Absolute Error: 1.6210131501730165
